In [1]:
import sklearn
import numpy as np
import pandas as pd
import collections

### Import Train and Test data

In [2]:
train_file = "data/TrainingSet.csv"
test_file = "data/TestSet.csv"

In [3]:
train_df = pd.read_csv(train_file, header = 0)
test_df = pd.read_csv(test_file, header = 0)

In [4]:
# hand picked features
# features = ["AuctionMedianPrice", "Price","PricePercent","StartingBidPercent","SellerClosePercent","StartingBid","AvgPrice","HitCount","AuctionAvgHitCount","ItemAuctionSellPercent","SellerSaleAvgPriceRatio","AuctionHitCountAvgRatio","BestOffer", "IsHOF","ItemListedCount","AuctionCount","AuctionSaleCount","SellerAuctionCount","SellerAuctionSaleCount"]
# features = ["AuctionMedianPrice", "Price", "AvgPrice", "ItemAuctionSellPercent", "StartingBidPercent", "StartingBid", "AuctionHitCountAvgRatio", "SellerSaleAvgPriceRatio", "IsHOF", "AuctionCount", "SellerAuctionSaleCount"]

# almost all features
features = ["Price","PricePercent","StartingBidPercent","SellerClosePercent","Category","StartingBid","AvgPrice","EndDay","HitCount","AuctionAvgHitCount","AuctionHitCountAvgRatio","BestOffer","AuctionCount","AuctionSaleCount","AuctionMedianPrice"]

# no endday
# features = ["Price","PricePercent","StartingBidPercent","SellerClosePercent","Category","StartingBid","AvgPrice","HitCount","AuctionAvgHitCount","AuctionHitCountAvgRatio","BestOffer","AuctionCount","AuctionSaleCount","AuctionMedianPrice"]

# took out a lot of features
# features = ["Price","StartingBid","AvgPrice","EndDay","HitCount","AuctionAvgHitCount","AuctionHitCountAvgRatio","BestOffer","AuctionCount","AuctionSaleCount","AuctionMedianPrice"]

In [5]:
len(test_df)
yes_test = test_df[test_df.QuantitySold == 1]
len(yes_test)

9399

### Parse out train data

In [6]:
import math
dayDict = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6}
# dayDict = {"Monday": math.sin(0), "Tuesday": math.sin(1), "Wednesday": math.sin(2), "Thursday": math.sin(3), "Friday": math.sin(4), "Saturday": math.sin(5), "Sunday": math.sin(6)}
train_df["EndDay"] = train_df["EndDay"].map(lambda d: dayDict[d])

In [7]:
sold = train_df[train_df.QuantitySold == 1]
not_sold = train_df[train_df.QuantitySold == 0]

In [8]:
len(sold), len(not_sold)

(79755, 178833)

In [9]:
even_train_df = pd.concat([sold, not_sold[:len(sold)]])
even_train_df.head()

,EbayID,QuantitySold,Price,PricePercent,StartingBidPercent,SellerName,SellerClosePercent,Category,PersonID,StartingBid,...,AuctionHitCountAvgRatio,BestOffer,ReturnsAccepted,IsHOF,ItemListedCount,AuctionCount,AuctionSaleCount,SellerAuctionCount,SellerAuctionSaleCount,AuctionMedianPrice
0,160983189073,1,27.25,1.4787,0.0537,petesandi,0.925926,73396,9174,0.99,...,29,1.540541,0,1,1,583,291,54,50,12.26
1,390549601720,1,43.00,1.4422,0.3350,graphn4fun,0.990566,27278,11046,9.99,...,55,0.404762,0,0,0,120,43,106,105,24.45
2,200901284676,1,11.00,0.5969,0.5426,hondo19461946,0.422131,73396,9174,10.00,...,21,0.324324,0,1,1,583,291,244,103,12.26
5,390551441929,1,33.01,0.8372,0.0251,firstclassautograph,0.664516,27276,22954,0.99,...,114,0.256098,0,1,0,47,23,155,103,41.00
9,121077510308,1,180.00,3.4830,3.4830,emmyjojo200011,0.333333,27278,20913,180.00,...,52,0.183333,0,0,1,1586,447,3,1,36.00


In [18]:
even_train_df = even_train_df.sample(frac=1)
even_train_df.head()

,EbayID,QuantitySold,Price,PricePercent,StartingBidPercent,SellerName,SellerClosePercent,Category,PersonID,StartingBid,...,AuctionHitCountAvgRatio,BestOffer,ReturnsAccepted,IsHOF,ItemListedCount,AuctionCount,AuctionSaleCount,SellerAuctionCount,SellerAuctionSaleCount,AuctionMedianPrice
77417,400450424738,1,14.99,2.6500,2.6500,bcsportscards,0.363833,73396,591,14.99,...,15,0.231884,0,1,0,6,3,2306,839,0.990
195216,121102095617,1,3.25,0.0645,0.0196,skyisthelimit3456,0.923077,27260,4407,0.99,...,48,0.163934,0,1,0,212,85,26,24,25.000
946,400447978454,0,0.99,0.8728,0.8728,the_sports_page,0.553381,73396,14129,0.99,...,4,0.069767,0,1,0,36,7,5264,2913,0.990
14885,261200847844,0,34.99,0.9322,0.9322,ksgmt,0.112825,27265,35460,34.99,...,43,0.263158,0,1,0,44,6,1037,117,22.995
109893,330882577069,0,120.00,2.5049,2.5049,yomongoni,0.063830,27278,26464,120.00,...,25,0.008696,0,1,1,618,198,94,6,33.775


In [19]:
X = even_train_df[features]
y = even_train_df["QuantitySold"]

### Build model

#### SVM

In [55]:
# from sklearn import svm

In [ ]:
# clf = svm.SVC(kernel="linear", C=0.025)

In [ ]:
# clf.fit(X, y)

#### Random Forest

In [ ]:
from sklearn import ensemble

In [ ]:
rf = ensemble.RandomForestClassifier(n_estimators=10)

In [ ]:
rf.fit(X, y)

#### KNN

In [ ]:
from sklearn import neighbors

In [ ]:
knn = neighbors.KNeighborsClassifier(5)

In [ ]:
knn.fit(X, y)

#### Decision Trees

In [ ]:
from sklearn import tree

In [ ]:
dt = tree.DecisionTreeClassifier()

In [ ]:
dt.fit(X, y)

#### Gaussian Naive Bayes

In [ ]:
from sklearn import naive_bayes

In [ ]:
gnb = naive_bayes.GaussianNB()

In [ ]:
gnb.fit(X, y)

#### ADA Boost

In [ ]:
ada = ensemble.AdaBoostClassifier()

In [ ]:
ada.fit(X, y)

#### MLP

In [20]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X)  
X_scaler = scaler.transform(X)  

In [21]:
from sklearn import neural_network

In [22]:
mlp = neural_network.MLPClassifier(solver='lbfgs')

In [23]:
mlp.fit(X_scaler, y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

### Parse Test Data

In [25]:
test_df["EndDay"] = test_df["EndDay"].map(lambda d: dayDict[d])

In [26]:
X_test = test_df[features]
y_test = test_df["QuantitySold"]

In [27]:
from sklearn import metrics

#### SVM Results

#### Random Forest

In [ ]:
rf_predicted = rf.predict(X_test)

In [ ]:
print(metrics.classification_report(rf_predicted, y_test))

#### KNN

In [ ]:
knn_predicted = knn.predict(X_test)

In [ ]:
print(metrics.classification_report(knn_predicted, y_test))

#### Decision Tree

In [ ]:
dt_predicted = dt.predict(X_test)

In [ ]:
print(metrics.classification_report(dt_predicted, y_test))

#### Gaussian Naive Bayes

In [ ]:
gnb_predicted = gnb.predict(X_test)

In [ ]:
print(metrics.classification_report(gnb_predicted, y_test))

#### ADA Boost

In [ ]:
ada_predicted = ada.predict(X_test)

In [ ]:
print(metrics.classification_report(ada_predicted, y_test))

#### MLP

In [28]:
X_test_scaler = scaler.transform(X_test)

In [29]:
mlp_predicted = mlp.predict(X_test_scaler)

In [30]:
print(metrics.classification_report(mlp_predicted, y_test))

             precision    recall  f1-score   support

          0       0.93      0.94      0.93     27728
          1       0.82      0.79      0.80      9732

avg / total       0.90      0.90      0.90     37460

